In [1]:
import pandas as pd
import numpy as np

In [5]:
# Read in NFL play-by-play data 2009-2018
df = pd.read_csv('NFLplays.csv', low_memory=False)

In [170]:
# Columns we'll use for machine learning
cols = ['game_date', 'game_id', 'posteam','defteam','drive','play_id', 
        'yardline_100', 'down', 'ydstogo', 'goal_to_go',
        'half_seconds_remaining', 'game_seconds_remaining', 'game_half',
        'shotgun', 'score_differential', 
        'qb_scramble', 'qb_dropback', 'play_type','rush_attempt', 'pass_attempt','sack'] 

# Columns we'll use to define pass vs rush plays
passcols = ['game_date','posteam', 'down', 'qb_scramble', 'qb_dropback', 
            'play_type','rush_attempt', 'pass_attempt', 'sack'] 

In [175]:
# Mask to keep data for 2017
# Remove January 1 game, that was part of the previous season!
mask2017season = (df.game_date.str.contains('2017')) & ~(df.game_date.str.contains('2017-01-01'))

# Sanity check to make sure we have a 16-game season for each team
test = df[cols][mask2017season]
test.groupby(['posteam'])['game_date'].nunique()   

# Just in case we need it again
df1 = df.copy()

df = df[cols][mask2017season]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45241 entries, 362094 to 407334
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_date               45241 non-null  object 
 1   game_id                 45241 non-null  int64  
 2   posteam                 43720 non-null  object 
 3   defteam                 43768 non-null  object 
 4   drive                   45241 non-null  int64  
 5   play_id                 45241 non-null  int64  
 6   yardline_100            43741 non-null  float64
 7   down                    38305 non-null  float64
 8   ydstogo                 45241 non-null  int64  
 9   goal_to_go              43789 non-null  float64
 10  half_seconds_remaining  45219 non-null  float64
 11  game_seconds_remaining  45219 non-null  float64
 12  game_half               45241 non-null  object 
 13  shotgun                 45241 non-null  int64  
 14  score_differential      43512 no

In [196]:
# Create masks for 1st thru 4th downs, run plays, and pass plays
downmask = ((df['down'] == 1) | (df['down'] == 2) | 
           (df['down'] == 3) | (df['down'] == 4))
df = df[downmask]

# A "run play" is any designed run (RB, QB, WR), but no QB scrambles or QB kneeldowns
# A "pass play" is any pass attempt (QB, RB, WR), and also QB sacks and QB scrambles
runmask = (df['play_type'] == 'run') & (df['qb_scramble'] == 0)
passmask = (((df['play_type'] == 'run') | (df['play_type'] == 'pass')) 
            & ((df['pass_attempt'] == 1) | (df['qb_dropback'] == 1)))

# Sanity check
df[passmask | runmask].count() == df[passmask].count() + df[runmask].count()


# Keep only run and pass columns, set PASS to 1 for passes and 0 for runs
df = df[passmask | runmask]
df['PASS'] = 0
df.loc[passmask,'PASS'] = 1

,game_id,drive,play_id,yardline_100,down,ydstogo,goal_to_go,half_seconds_remaining,game_seconds_remaining,shotgun,score_differential,qb_scramble,qb_dropback,rush_attempt,pass_attempt,sack,PASS
count,3.188400e+04,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000,31884.000000
mean,2.017108e+09,11.798802,2070.609459,52.648131,1.802471,8.703394,0.054196,849.574834,1748.163468,0.589104,-1.406442,0.022488,0.603563,0.418925,0.581075,0.037448,0.603563
std,1.107686e+04,6.930413,1218.771898,24.169714,0.819920,3.950816,0.226409,541.451688,1044.780935,0.492004,10.337054,0.148266,0.489165,0.493391,0.493391,0.189860,0.489165
min,2.017091e+09,1.000000,48.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-44.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.017101e+09,6.000000,1010.000000,34.000000,1.000000,6.000000,0.000000,357.000000,843.000000,0.000000,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.017111e+09,12.000000,2075.000000,57.000000,2.000000,10.000000,0.000000,842.000000,1800.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,2.017121e+09,17.000000,3100.000000,73.000000,2.000000,10.000000,0.000000,1321.000000,2638.000000,1.000000,4.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,2.017123e+09,33.000000,5531.000000,99.000000,4.000000,40.000000,1.000000,1800.000000,3600.000000,1.000000,44.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [199]:
df.PASS.value_counts()

1    19244
0    12640
Name: PASS, dtype: int64

In [218]:
df.to_csv('NFLplays2017.csv',index=False)